In [ ]:
import os
import random
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from termcolor import colored

import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
# Path of dataset
base_dir = './datasets'
# base_dir = '/kaggle/input/playing-cards-object-detection-dataset'

# Path of yaml file
yaml_path = os.path.join(base_dir, 'data.yaml')

# Path of Train directory
train_dir = os.path.join(base_dir, 'train')
train_images = os.path.join(train_dir, 'images')

# Path of Validation directory
valid_dir = os.path.join(base_dir, 'valid')
valid_images = os.path.join(valid_dir, 'images')

# Path of Test directory
test_dir = os.path.join(base_dir, 'test')
test_images = os.path.join(test_dir, 'images')

In [ ]:
# All the classes from yaml file
class_names = ['10c', '10d', '10h', '10s', '2c', '2d', '2h', '2s', '3c', '3d', '3h', '3s', '4c', '4d', '4h', '4s', '5c', '5d', '5h', '5s', '6c', '6d', '6h', '6s', '7c', '7d', '7h', '7s', '8c', '8d', '8h', '8s', '9c', '9d', '9h', '9s', 'Ac', 'Ad', 'Ah', 'As', 'Jc', 'Jd', 'Jh', 'Js', 'Kc', 'Kd', 'Kh', 'Ks', 'Qc', 'Qd', 'Qh', 'Qs']
print(colored(f'Number of Classes : {len(class_names)}', 'green', attrs=['bold']))

In [ ]:
print(colored(f'Number of Train Images : {len(os.listdir(train_images))}', 'blue', attrs=['bold']))
print(colored(f'Number of Validation Images : {len(os.listdir(valid_images))}', 'blue', attrs=['bold']))
print(colored(f'Number of Test Images : {len(os.listdir(test_images))}', 'blue', attrs=['bold']))

In [ ]:
# Define a set to store unique sizes
size = set()

# Loop over train images to check image sizes
for img_name in os.listdir(train_images) :
    img_path = os.path.join(train_images, img_name)
    img = cv2.imread(img_path)
    h, w, c = img.shape
    size.add((h, w, c))

if len(size) == 1 :
    print(colored(f'All of images have size of {(w, h, c)}', 'green', attrs=['bold']))
else :
    print(colored(f'There are {len(size)} different image sizes !', 'red', attrs=['bold']))

In [ ]:
img_size = (h, w)

In [ ]:
translate_cards = {
    's' : 'Spades',
    'c' : 'Clubs',
    'h' : 'Hearts',
    'd' : 'Diamonds',
    'A' : 'Ace',
    'K' : 'King',
    'Q' : 'Queen',
    'J' : 'Jack'
}


In [ ]:
def label_translator(labels) :
    ''' Translate chars to labels '''
    translated_label = ''

    for card in labels :

        if card[0] in ['A', 'K', 'Q', 'J'] :
            c0 = translate_cards[card[0]]
        else :
            if len(card) == 2 :
                c0 = card[0]
            elif len(card) == 3 :
                c0 = card[:2]

        card = c0 + '-' + translate_cards[card[-1]]

        translated_label += card + '\n'
        
    translated_label = translated_label[:-1]
        
    return translated_label

In [ ]:
def Draw_Bounding_Boxes(img_name) :
    ''' Show image / Draw boxes / Write class label '''
    img_path = os.path.join(train_images, img_name)
    img_path

    yaml_name = img_name[:-4] + '.txt'
    yaml_path = os.path.join(os.path.join(train_dir, 'labels'), yaml_name)

    title = []

    font = cv2.FONT_HERSHEY_SIMPLEX

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


    f1 = open(yaml_path, "r")
    f2 = open(yaml_path, "r")

    for i in range(len(f1.readlines())) :
        text = f2.readline()

        yaml = text.split()

        class_name = class_names[int(yaml[0])]
        if class_name not in title :
            title.append(class_name)


        x_center = int(float(yaml[1]) * 416)
        y_center = int(float(yaml[2]) * 416)
        w = int(float(yaml[3]) * 416)
        h = int(float(yaml[4]) * 416)

        x1 = x_center - int(w/2)
        y1 = y_center - int(h/2)

        x2 = x_center + int(w/2)
        y2 = y_center + int(h/2)

        # Draw rectangles and text on image
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(img, (x1+5, y1-35), (x1+65, y1-5), (255, 255, 255), -1, cv2.LINE_AA)
        cv2.putText(img, class_name, (x1+10, y1-10), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    title = label_translator(title)
    plt.title(title, fontsize=10, fontweight='bold')
    plt.axis('off')
    plt.imshow(img)

In [ ]:
images = os.listdir(train_images)
num_samples = 15

random_images = random.choices(images, k=num_samples)

plt.figure(figsize=(12, 8), dpi=200)
for i in range(15) :
    plt.subplot(3, 5, i+1)
    Draw_Bounding_Boxes(random_images[i])
plt.show()


In [ ]:
# Load a row model and load trained weights to it
model = YOLO('yolov8n.yaml').load('yolov8n.pt')


# Train the model
result = model.train(data=yaml_path, epochs=150, imgsz=img_size[0])

In [ ]:
# Location of stored result
result_path = result.save_dir

In [ ]:
# List of some metrics
metrics = ['results.png','confusion_matrix.png', 'P_curve.png', 'PR_curve.png']

plt.figure(figsize=(15, 12))
i = 1

for image_name in metrics:
    image_path = os.path.join(result_path, image_name)
    image = cv2.imread(image_path)
    
    plt.subplot(2, 2, i)
    
    plt.imshow(image)
    plt.title(image_name)
    i += 1

plt.show()

In [ ]:
# Read csv of result
df = pd.read_csv(os.path.join(result_path, 'results.csv'))
df.tail(5)

In [ ]:
# Strip columns
df.columns = df.columns.str.strip()
df.columns

In [ ]:
# Define X, y to plot
X = df.epoch
cols = df.columns[1:]

plt.figure(figsize=(15, 25))

for it in range(len(cols)) :
    plt.subplot(7, 2, it+1)
    ax = sns.lineplot(data=df, x=X, y=cols[it])
    ax.set_title(cols[it])
plt.suptitle('Training Metrics and Loss', fontsize=24)
plt.subplots_adjust(top=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Load model with best weights
best_model_path = os.path.join(result_path, 'weights/best.pt')

best_model = YOLO(best_model_path)

In [ ]:
# Evaluating the model
result_test = best_model.val()

In [ ]:
result_test

In [ ]:
keys = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']

for key in keys :
    print(colored(f'{key} : {result_test.results_dict[key]}', 'green', attrs=['bold']))

### Test predictions

In [ ]:
from PIL import Image

# test_img = "labai1.jpg"
test_img = "./datasets/test/images/2024-08-12-22-35-03_mp4-0007_jpg.rf.eda4a8f142fed9363991917a420ede69.jpg"
img1 = Image.open(test_img)


result = best_model.predict([img1], save=True, conf=0.1, iou=0.7)